In [3]:
# 导入必要的库
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain import hub
from langchain_openai import ChatOpenAI


In [4]:
# 步骤1: 加载离线文档
loader = TextLoader("/home/zhangbaoxing/openai-quickstart/langchain/jupyter/LCEL/031_Humorous_method.md")
docs = loader.load()

In [5]:
# 检查加载的文档内容长度
print(len(docs[0].page_content))  # 打印第一个文档内容的长度

3754


In [7]:
# 查看第一个文档（前100字符）
print(docs[0].page_content[:100])

---
title: 31_幽默的方法
author: Baoxing
time: 2024-06-14 09:04:22
tags:
  - 学习
  - 精英日课
  - 万维刚
  - 幽默
 


In [8]:
# 步骤2: 文档分割
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)

In [9]:
# 检查分割后的块数量和内容
print(len(all_splits))  # 打印分割后的文档块数量

5


In [10]:
print(len(all_splits[0].page_content))  # 打印第一个块的字符数

972


In [11]:
print(all_splits[0].page_content)  # 打印第一个块的内容

---
title: 31_幽默的方法
author: Baoxing
time: 2024-06-14 09:04:22
tags:
  - 学习
  - 精英日课
  - 万维刚
  - 幽默
  - 精英日课第6季
rating: -1
---
谁不想做个幽默的人呢？跟人初次见面，你三言两语就能把人逗笑，你立即就会受到欢迎。做个报告，中间穿插几个幽默的点，你讲的东西就更容易被接受。领导讲话，加几个接地气的笑话，更能取得观众支持。

2018年，第41任美国总统老布什去世。他的儿子小布什，也就是第43任美国总统，在追思会上发表了一个很正式的追忆讲话。而就在这么一个悲伤的场合，他还是讲了几个笑话。当然那是笑中带泪，但你能体会到幽默哪怕在这样的情境中也有好效果。

现代社会的一个几乎默认的社交礼仪，就是如果有一屋子人听你一个人讲话，你最好能穿插几个笑话给人提供一点娱乐。幽默既能让现场氛围有松弛感，又能吸引注意，特别适合发挥影响力。

但怎么才能做到幽默呢？

✵

我理解「幽默」是个很近代的概念。咱们中国自古以来就有专门的「笑话」集，有喜剧表演，但是没上升到幽默的认识。一直到1924年，才由林语堂先生把英文 Humor 这个词翻译成幽默。

幽默是比笑话更广义的一种表达方式。笑话是一定要讲个小故事，而幽默则可以用动作、表情、行为表现，你还可以仅仅用一个场景设定、甚至是一些无人的元素展现幽默，你可以用幽默的方式讲一个严肃的话题。幽默可以是一个佐料，可以无缝添加到各个场景之中，立即就给那个场景增加一个会让人笑的色彩。

每个人都把别人逗乐过，但你要说人到底是怎么乐的，那真是很难说。很多时候幽默像是灵机一动的神来之笔，又像是变魔术一样：我们只看到结果，没明白背后的机制是什么。

林语堂说：「幽默是一种人生的观点，一种应付人生的方法。幽默没有旁的，只是智慧之刀的一晃。」

可那一刀是怎么晃的呢？有没有什么系统性的制造幽默的方法呢？

老百姓有些寻常的说法，都不怎么靠谱。有人说幽默就是“看坏事发生在别人身上”。就如同有人踩西瓜皮摔倒了，你在旁边哈哈大笑。还有人说幽默就是“悲剧加上时间”，也许有些坏事儿时间长了就没有那么坏，就可以让人笑了。可是我们明明知道，有很多幽默场景 —— 特别是高级的幽默场景 —— 是没有坏事的啊。


In [12]:
# 步骤3: 存储嵌入
vectorstore = Chroma.from_documents(
    documents=all_splits,
    embedding=OpenAIEmbeddings()
)

In [13]:
# 步骤4: 检索文档
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

In [21]:
# 步骤5: 生成回答
llm = ChatOpenAI(model="gpt-4o-mini")
prompt = hub.pull("rlm/rag-prompt")

/root/miniconda3/envs/langchain/lib/python3.10/site-packages/langchain/hub.py:86: DeprecationWarning: The `langchainhub sdk` is deprecated.
Please use the `langsmith sdk` instead:
  pip install langsmith
Use the `pull_prompt` method.
  res_dict = client.pull_repo(owner_repo_commit)


In [26]:
def format_docs(docs):
    # print('*'*50)
    # print( "\n\n".join(doc.page_content for doc in docs))
    # print('*'*50)
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


In [27]:
# 流式生成回答
for chunk in rag_chain.stream("怎样变得幽默"):
    print(chunk, end="", flush=True)

Number of requested results 6 is greater than number of elements in index 5, updating n_results = 5


要变得幽默，可以尝试打破常规的模式，制造意外的幽默效果。观察生活中的细节，识别并强化一些常见的偏见或刻板印象，也能产生幽默感。多加练习和总结这些技巧，有助于提升自己的幽默感。

In [16]:
# 流式生成回答
for chunk in rag_chain.stream("幽默的方式"):
    print(chunk, end="", flush=True)

Number of requested results 6 is greater than number of elements in index 5, updating n_results = 5


幽默可以通过打破常规模式、发现新模式或强化刻板印象来实现。比如，通过在意想不到的场合中引入出乎意料的角色或行为，能够制造幽默效果。同时，幽默是一种观察生活的方式，帮助我们超越平庸，增添乐趣。

In [17]:
# 测试三个问题
questions = [
    "什么是幽默?",
    "如何练习幽默?",
    "如何制造幽默?"
]

for question in questions:
    print(f"问题: {question}")
    print("回答:", rag_chain.invoke(question))
    print("\n")

问题: 什么是幽默?


Number of requested results 6 is greater than number of elements in index 5, updating n_results = 5


回答: 幽默是一种比笑话更广泛的表达方式，既可以通过语言、动作或场景展现，也可以用于轻松应对严肃话题。它被视为一种智慧的体现，能够打破常规和模式，引发人们的笑声。幽默不仅增加社交吸引力，还能缓解紧张氛围，促进交流。


问题: 如何练习幽默?


Number of requested results 6 is greater than number of elements in index 5, updating n_results = 5


回答: 练习幽默可以通过观察生活中的模式并找出意外的反转来实现。可以尝试打破常见的模式，发现新的幽默场景，或者强化一些刻板印象。多加练习和观察，幽默感会逐渐增强。


问题: 如何制造幽默?


Number of requested results 6 is greater than number of elements in index 5, updating n_results = 5


回答: 制造幽默可以通过打破常规模式、发现新模式或强化偏见来实现。例如，打破大家对某种场景的预期，比如一个强壮的男人进入针织俱乐部学习织毛衣，就会产生幽默效果。此外，幽默的本质在于轻微的违反，比如违反社会习俗或逻辑，能够引发人们的笑声。


